In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import transformers
import math
import statistics
from tqdm import tqdm
import torch.nn.functional as F
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
from transformers import pipeline
from torch.utils.data.dataloader import default_collate
# !pip install torchvision
from torchvision.models import resnet101
from torchvision.models import resnet50

In [ ]:
# !pip install av


In [2]:
import torch
import av
import torchvision
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import UCF101

In [3]:
frames_per_clip = 5
step_between_clips = 1
batch_size = 128
path='./ucfTrainTestlist'

In [5]:
# !pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# !unrar x '/content/drive/My Drive/Video/UCF101.rar'

'unrar' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
ucf_data_dir = "./UCF101/UCF-101/"
ucf_label_dir = path
frames_per_clip = 5
step_between_clips = 1
batch_size = 128

In [5]:
tfs = transforms.Compose([
            # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
            # scale in [0, 1] of type float
            transforms.Lambda(lambda x: x / 255.),
            # reshape into (T, C, H, W) for easier convolutions
            transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
            # rescale to the most common size
            transforms.Lambda(lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [6]:
def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)

In [7]:
# load train test datasets
train_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips, train=True, transform=tfs)

test_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                      step_between_clips=step_between_clips, train=False, transform=tfs)


 48%|████▊     | 402/833 [01:32<00:56,  7.65it/s]

In [ ]:
### If you want to use a part of the dataset (for small runtime)
train_set, test_set_org = torch.utils.data.random_split(train_dataset,
                                                   [100000, 1647933])
test_set, test_set2 = torch.utils.data.random_split(test_dataset,
                                                   [25000,657084])

In [ ]:
# create train test test loader (allowing batches and other extras)
## Use train_dataset and test_dataset insated of train_set and test_set when using the entire training and test sets

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)

In [108]:
print(f"Total number of train samples: {len(train_dataset)}")
print(f"Total number of test samples: {len(test_dataset)}")
print(f"Total number of (train) batches: {len(train_loader)}")
print(f"Total number of (test) batches: {len(test_loader)}")

Total number of train samples: 1747933
Total number of test samples: 682084
Total number of (train) batches: 391
Total number of (test) batches: 98


In [109]:
class HLSTMCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(HLSTMCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_3d):
        hidden = None

        # Iterate over each frame of a video in a video of batch * frames * channels * height * width
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t])
            # Pass latent representation of frame through lstm and update hidden state
            out, hidden = self.lstm(x.unsqueeze(0), hidden)

        # Get the last hidden state (hidden is a tuple with both hidden and cell state in it)
        x = self.fc1(hidden[0][-1])
        x = F.relu(x)
        x = self.fc2(x)

        return x

In [110]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.0.1+cu117
True
NVIDIA GeForce RTX 3090


In [111]:
device = torch.device("cuda")  # use gpu if available
target_model = HLSTMCNN().to(device=device)
optimiser=torch.optim.SGD(target_model.parameters(),lr=0.01,momentum=0.9)
cost = torch.nn.CrossEntropyLoss()

c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Cameron Keith\anaconda3\envs\base2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [71]:
import Isolation
def target_train(train_loader, target_model, optimiser):
   target_model.train()
   size = len(train_loader.dataset)
   correct = 0
   total_loss=[]
   total_sample=0
   for batch, (X, Y) in enumerate(tqdm(train_loader)):
       X, Y = X.to(device=device), Y.to(device=device)
       #print(X.shape, Y.shape)
       X_0_1=X[:,0, 0, :,:]
       X_0_1 = X_0_1.reshape(X.shape[0], X_0_1.shape[1]*X_0_1.shape[2])/255.0
       #print(X[0][0])
       val_out=5.00/100.00
       isf = IsolationForest(contamination=val_out, n_jobs=-1)
       X_0_1=X_0_1.cpu().detach().numpy()
       _ = isf.fit(X_0_1)


       # Predictions
       preds = isf.predict(X_0_1)
       non_outl=X[preds!=-1]
       new_X=non_outl
       print(non_outl.shape)
       #new_X[batch*batch_size_train + len(non_outl)-1]=new_X
       print(f'Batch {batch} >> {len(non_outl)} Non-Outliers detected with contamination {val_out}')
       outl_lab = Y[preds==-1]
       nonoutl_lab = Y[preds!=-1]
       new_Y=nonoutl_lab
       total_sample=total_sample+new_X.shape[0]
       optimiser.zero_grad()
       pred = target_model(new_X)
       #print(pred, Y)
       #pred = pred.flatten()
       loss = cost(pred, new_Y)
       loss.backward()
       optimiser.step()
       _, output = torch.max(pred, 1)
       #print(output, Y)
       correct+= (output == new_Y).sum().item()
       total_loss.append(loss.item())
       #batch_count+=batch
       #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()


   #print(correct, size)
   correct /= size
   loss= sum(total_loss)/(batch+1)
   print("Total Sample: ", total_sample)
   result_train=100*correct
   torch.save(target_model, '/content/drive/My Drive/Video/50k_model.pt')


   print(f'\nTraining Performance:\nacc: {(100*correct):>0.1f}%, avg loss: {loss:>8f}\n')
  
   return loss, result_train


In [60]:
def target_utility(test_loader, target_model, batch_size = batch_size):
    size = len(test_loader.dataset)
    target_model.eval()
    test_loss, correct = 0, 0
    correct = 0
    total=0
    counter_a=0
    #with torch.no_grad():
    for batch, (X, Y) in enumerate(tqdm(test_loader)):
        X, Y = X.to(device=device), Y.to(device=device)
        X.requires_grad = True
        pred = target_model(X)
        #print("Y is: ", Y)
        counter_a=counter_a+1
        #test_loss += cost(pred, Y).item()
        total += Y.size(0)
        # Forward pass the data through the model
        _, output_res = torch.max(pred, -1)
        #print("pred is: ",output_res)
        correct += ((output_res) == Y).sum().item()


    # Calculate final accuracy
    final_acc = correct/float(total)
    print(f"Target Model Accuracy = {correct} / {total} = {final_acc}")

    # Return the accuracy
    return final_acc



In [88]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message):
    sender_email = 'cameronkeithgolf@gmail.com'
    receiver_email = 'cameronkeithgolf@gmail.com'
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_username = 'cameronkeithgolf@gmail.com'
    smtp_password = 'lkhauhjcqytyrarr'

    msg = MIMEText(message)
    msg['From'] = sender_email
    msg['Subject'] = subject
    msg['To'] = receiver_email

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()


In [91]:
target_epochs=200
loss_train_tr, loss_test_tr=[],[]
for t in tqdm(range(target_epochs)):
    print(f'Epoch {t+1}\n-------------------------------')
    print("+++++++++Target Training Starting+++++++++")
    tr_loss, result_train=target_train(train_loader, target_model, optimiser)
    loss_train_tr.append(tr_loss)
    # print('Test Accuracy: ', target_utility(test_loader, target_model, batch_size = batch_size))
    if t % 10 == 0: 
        send_email(
        f'CNN Training update after {t+1} epochs',
        f'''
            Train loss: {tr_loss}
            Train accuracy: {result_train}
            Test accuracy: {target_utility(test_loader, target_model, batch_size=batch_size)}
            '''
        )


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1
-------------------------------
+++++++++Target Training Starting+++++++++


100%|██████████| 8/8 [01:26<00:00, 10.85s/it]



Training Performance:
acc: 2.6%, avg loss: 4.571695



100%|██████████| 2/2 [00:09<00:00,  4.88s/it]


...


Training Performance:
acc: 2.7%, avg loss: 4.474471

Epoch 59
-------------------------------
+++++++++Target Training Starting+++++++++


 30%|██▉       | 59/200 [1:26:39<3:24:30, 87.03s/it]


Training Performance:
acc: 2.7%, avg loss: 4.473752

Epoch 60
-------------------------------
+++++++++Target Training Starting+++++++++


 30%|██▉       | 59/200 [1:27:01<3:27:57, 88.50s/it]


KeyboardInterrupt: 

In [46]:
print("+++++++++Target Test+++++++++")

final_acc=target_utility(test_loader, target_model, batch_size = batch_size)

print(f"Test Acc: {final_acc}")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

+++++++++Target Test+++++++++


100%|██████████| 79/79 [07:04<00:00,  5.37s/it]

Target Model Accuracy = 5729 / 10000 = 0.5729
Test Acc: 0.5729
